In [262]:
import os, sys
sys.path.insert(0, '/Users/shashank/Workspace/Orgs/Ether/ai-engine/services/recommendations')

In [263]:
%load_ext autoreload
%autoreload 2

In [264]:
import numpy as np
import json as js
from scipy.spatial.distance import cosine

In [ ]:
from vectorize import Vectorizer
from lsh import Table, LSH, Search, MinHash
from graph_query import DgraphClient

In [266]:
url = "111.93.155.194:9080"

In [267]:
def to_json(data, filename):
    with open(filename + ".json", "w", encoding="utf-8") as f_:
        js.dump(data, f_, ensure_ascii=False, indent=4)

def read_json(json_file):
    with open(json_file) as f_:
        meeting = js.load(f_)
    return meeting

In [268]:
user_kw_query = """
query userKw($n: string, $t: int) {
    userKw(func: eq(email, $n)) {
        uid
        xid
        name
        mentionName
        ~hasMember{
            name
        }
        ~authoredBy (first: $t) {
            hasKeywords {
                values
            }
        }
    }
}
"""

In [272]:
vectorizer = Vectorizer()
dg = DgraphClient(url=url)

In [273]:
variables = {"$n": "shashank@etherlabs.io", "$t": "5"}
user_kw_resp = dg.perform_query(query=user_kw_query, variables=variables)

_Rendezvous: <_Rendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "Operation timed out"
	debug_error_string = "{"created":"@1574398856.746237000","description":"Error received from peer","file":"src/core/lib/surface/call.cc","file_line":1039,"grpc_message":"Operation timed out","grpc_status":14}"
>

In [148]:
user_kw_resp

{'userKw': [{'uid': '0xcfc87',
   'xid': 'e034c7a6-0681-40ba-b7b2-b78de2a2cc3d',
   'name': 'Shashank',
   'mentionName': 'shashank',
   '~authoredBy': [{'hasKeywords': [{'values': ['CSV ether ready',
        'transportation app']}]},
    {'hasKeywords': [{'values': ['animation to switched',
        'HTTP libraries',
        'HTTP',
        'real change white code',
        'second time',
        'offline mode',
        'images just renders',
        'app remember',
        'catch data',
        'jump offline',
        'choose is random']}]},
    {'hasKeywords': [{'values': ['great examples',
        'Lambda',
        'modern application',
        'Lambda functions',
        'lot of different Lambda functions']}]},
    {'hasKeywords': [{'values': ['real change',
        'web component',
        'app Remember',
        'HTTP libraries',
        'second time round',
        'start the app',
        'footer colleagues javascript',
        'couple of differences',
        'chose is a rando

In [158]:
def format_response(resp):
    user_id_dict = {}
    user_kw_dict = {}
    for info in resp["userKw"]:
        try:
            user_id = info["xid"]
            user_name = info["name"]

            user_id_dict.update({
                user_id: user_name
            })

            keyword_object = info["~authoredBy"]
            user_kw_list = []
            for obj in keyword_object:
                kw = obj["hasKeywords"]
                user_kw_list.extend([words for words in kw[0]["values"]])

            user_kw_dict.update({
                user_id: user_kw_list
            })
        except Exception:
            continue
    
    return user_kw_dict, user_id_dict

In [174]:
user_kw_dict, user_id_dict = format_response(resp=user_kw_resp)

In [224]:
user_kw_dict

{'e034c7a6-0681-40ba-b7b2-b78de2a2cc3d': ['CSV ether ready',
  'transportation app',
  'animation to switched',
  'HTTP libraries',
  'HTTP',
  'real change white code',
  'second time',
  'offline mode',
  'images just renders',
  'app remember',
  'catch data',
  'jump offline',
  'choose is random',
  'great examples',
  'Lambda',
  'modern application',
  'Lambda functions',
  'lot of different Lambda functions',
  'real change',
  'web component',
  'app Remember',
  'HTTP libraries',
  'second time round',
  'start the app',
  'footer colleagues javascript',
  'couple of differences',
  'chose is a random',
  'Stitches the same styling',
  'web app',
  'HTTP',
  'Switched Off',
  'working hard',
  'deploy by cash',
  'offline mode',
  'core complex pieces',
  'switch up online'],
 'be748cf6-d983-4ac1-8854-97fdc98c056d': ['preference relations',
  'variables by refer',
  'ends the NI belong',
  'training and test',
  'architecture like work',
  'carry on context',
  'idea of gener

In [176]:
ref_user_kw_query = """
query userKw($t: int) {
    userKw(func: type("User")) {
        uid
        xid
        name
        mentionName
        ~hasMember{
            name
        }
        ~authoredBy (first: $t) {
            hasKeywords {
                values
            }
        }
    }
}
"""

In [177]:
var = {"$t": "5"}
ref_user_kw_resp = perform_query(query=ref_user_kw_query, variables=var)

In [178]:
ref_user_kw_resp

{'userKw': [{'uid': '0xa633a',
   'xid': '18b15cf4-bcc7-4b0a-8d46-8a05421188c7',
   'name': 'F1 Testing',
   'mentionName': 'f1'},
  {'uid': '0xa6387',
   'xid': '3a23aba9-7d04-4451-927c-684c1673496e',
   'name': 'Franklin Ferdinand',
   'mentionName': 'franklin',
   '~authoredBy': [{'hasKeywords': [{'values': ['Keys']}]},
    {'hasKeywords': [{'values': ['nothingto Those people', 'Theo', 'Over']}]},
    {'hasKeywords': [{'values': ['marks Jira',
        'civil rights Isis',
        'open project',
        'climate changes',
        'climate nats',
        'lot of salt',
        'smooth change',
        'wait Meetings',
        'speak aloud',
        'feel ability',
        'Redis',
        'Isis',
        'ability to speak aloud']}]},
    {'hasKeywords': [{'values': ['process refund',
        'process purchase',
        'basic role',
        'great function',
        'leave a default option create role',
        'create our two lambda channel',
        'AWS Console',
        'basic la

In [179]:
ref_user_kw_dict, ref_user_id_dict = format_response(resp=ref_user_kw_resp)

In [180]:
len(ref_user_id_dict.keys())

179

In [181]:
len(ref_user_kw_dict)

23

In [182]:
ref_user_kw_dict

{'3a23aba9-7d04-4451-927c-684c1673496e': ['Keys',
  'nothingto Those people',
  'Theo',
  'Over',
  'marks Jira',
  'civil rights Isis',
  'open project',
  'climate changes',
  'climate nats',
  'lot of salt',
  'smooth change',
  'wait Meetings',
  'speak aloud',
  'feel ability',
  'Redis',
  'Isis',
  'ability to speak aloud',
  'process refund',
  'process purchase',
  'basic role',
  'great function',
  'leave a default option create role',
  'create our two lambda channel',
  'AWS Console',
  'basic lambda',
  'Lambda dot',
  'guys called the process'],
 'feffe3ee-896f-45c3-9d91-1f6d962c66ce': ['insert events',
  'lambda arguments',
  'add some more criteria',
  'combinations of criterias',
  'change events',
  'players scores',
  'lot is data replication',
  'record as the argument',
  'guy to this',
  'table at real time',
  'player ID play',
  'replicate dot',
  'bit more searchable',
  'leaderboards a tree',
  'insert on the table',
  'batch of changes',
  'elastic search cl

In [183]:
modified_ref_user_id_dict = {k: v for k, v in ref_user_id_dict.items() if k in ref_user_kw_dict.keys()}

In [184]:
len(modified_ref_user_id_dict)

23

## Remove users (if any) that are being compared against

In [185]:
for k in modified_ref_user_id_dict.keys():
    if k in user_kw_dict.keys():
        del ref_user_kw_dict[k]

In [186]:
len(ref_user_kw_dict)

21

In [189]:
aws_config = Config(
        connect_timeout=180,
        read_timeout=300,
        retries={"max_attempts": 2},
        region_name="us-east-1",
    )
lambda_client = bclient("lambda", config=aws_config)
lambda_function = "sentence-encoder-lambda"

def get_embeddings(input_list):
    lambda_payload = {"body": {"text_input": input_list}}

    try:
        invoke_response = lambda_client.invoke(
            FunctionName=lambda_function,
            InvocationType="RequestResponse",
            Payload=js.dumps(lambda_payload),
        )

        lambda_output = (
            invoke_response["Payload"].read().decode("utf8").replace("'", '"')
        )
        response = js.loads(lambda_output)
        status_code = response["statusCode"]
        response_body = response["body"]

        if status_code == 200:
            embedding_vector = np.asarray(js.loads(response_body)["embeddings"])

        else:
            embedding_vector = np.asarray(js.loads(response_body)["embeddings"])

        return embedding_vector

    except Exception as e:
        print(e)
        embedding_vector = np.zeros((1, 512))
        return embedding_vector

In [190]:
vec1 = get_embeddings(input_list=["AWS lambda function"])
vec2 = get_embeddings(input_list=["AWS serverless computing"])

In [191]:
sim = 1 - cosine(vec1, vec2)

In [192]:
sim

0.7383127021592407

In [206]:
def hash_func(vecs, projections):
    bools = np.dot(vecs, projections.T) > 0
    return [bool2int(bool_vec) for bool_vec in bools]

def bool2int(x):
    y = 0
    for i,j in enumerate(x):
        if j: y += 1<<i
    return y

In [252]:
class Table:
    
    def __init__(self, hash_size, dim):
        self.table = dict()
        self.hash_size = hash_size
        self.projections = np.random.randn(self.hash_size, dim)

    def add(self, vecs, label):
        entry = {'label': label}
        hashes = hash_func(vecs, self.projections)
        for h in hashes:
            if h in self.table.keys():
                self.table[h].append(entry)
            else:
                self.table[h] = [entry]

    def query(self, vecs):
        hashes = hash_func(vecs, self.projections)
        results = list()
        for h in hashes:
            if h in self.table.keys():
                results.extend(self.table[h])
        return results

class LSH:
    
    def __init__(self, dim):
        self.num_tables = 4
        self.hash_size = 8
        self.tables = list()
        for i in range(self.num_tables):
            self.tables.append(Table(self.hash_size, dim))
    
    def add(self, vecs, label):
        for table in self.tables:
            table.add(vecs, label)
    
    def query(self, vecs):
        results = list()
        for table in self.tables:
            results.extend(table.query(vecs))
        return results

    def describe(self):
        for table in self.tables:
            print(table.table)

class Search:
    
    def __init__(self, input_dict):
        self.dim_size = 512
        self.lsh = LSH(self.dim_size)
        self.input = input_dict
        self.num_features_in_input = dict()
        for f, kw in self.input.items():
            self.num_features_in_input[f] = 0
                
    def featurize(self):
        for user, kw_list in self.input.items():
            kw_features = get_embeddings(input_list=kw_list)
            self.lsh.add(kw_features, user)
            self.num_features_in_input[user] += len(kw_features)
                
    def query(self, user_dict, user):
        kw_list = user_dict[user]
        kw_features = get_embeddings(input_list=kw_list)
        
        results = self.lsh.query(kw_features)
        print('num results', len(results))

        counts = dict()
        for r in results:
            if r['label'] in counts.keys():
                counts[r['label']] += 1
            else:
                counts[r['label']] = 1
        for k in counts:
            counts[k] = float(counts[k])/self.num_features_in_input[k]
        return counts

In [253]:
ws = Search(input_dict=ref_user_kw_dict)

In [254]:
ws.featurize()

In [255]:
ws.num_features_in_input

{'3a23aba9-7d04-4451-927c-684c1673496e': 27,
 'feffe3ee-896f-45c3-9d91-1f6d962c66ce': 94,
 'f8dd2818-fb6e-450b-bfe4-73ae75ae94c2': 8,
 '708017d9-e0da-4f67-96d2-06ac853aaffc': 17,
 'b1b92672-20ce-4e0c-af0a-82de3016fcb0': 17,
 'd5934fd1-809a-47d9-9ed8-fa81b22f3657': 105,
 '7833fafc-39bb-4ea8-8f39-348e9e536390': 40,
 'a366bdbc-ba2f-4b09-ba6c-ad0c614af9ca': 116,
 'edd78ba4-b452-4816-8f3d-0886cba74f40': 144,
 '2cd90f06-74f3-48cc-922a-cd6b8782ba0f': 60,
 'cd4e688d-9011-470a-8e4b-674cd3b40977': 5,
 'e5bdca94-c037-4a67-bcb8-350efeae05bd': 28,
 '31a3ba47-6185-4ad9-a041-ddf1c4c6a1dc': 146,
 '85dd59a9-7cf8-49a2-abc4-32b5027a6147': 30,
 'c2603593-2d62-4f68-b615-d2cef2145608': 50,
 '53ef4e1c-8621-47f5-a0b3-9e664ffd7170': 6,
 'ad57d007-5271-410d-aa0a-bcb4b0186639': 2,
 '7e701eae-c283-4df5-abfa-4c0fcb3d7a94': 24,
 '25f4a8bf-6e75-46b5-b42c-b4be8808b217': 27,
 'ead1d4d7-56a0-4584-8c31-7aa2d41bed49': 21,
 '88e54601-33d0-4b41-ad9b-79e5a7edb957': 3}

In [259]:
result = ws.query(user_dict=user_kw_dict, user='e034c7a6-0681-40ba-b7b2-b78de2a2cc3d')

num results 2450


In [260]:
for r in sorted(result, key=result.get, reverse=True):
    print(r, ref_user_id_dict[r], result[r])

88e54601-33d0-4b41-ad9b-79e5a7edb957 F1 Testing 4.0
708017d9-e0da-4f67-96d2-06ac853aaffc Vamshi 3.176470588235294
53ef4e1c-8621-47f5-a0b3-9e664ffd7170 Krishna Sai 3.1666666666666665
f8dd2818-fb6e-450b-bfe4-73ae75ae94c2 Nikita Sinha 3.125
b1b92672-20ce-4e0c-af0a-82de3016fcb0 mithun 3.1176470588235294
7e701eae-c283-4df5-abfa-4c0fcb3d7a94 Vani 3.0
d5934fd1-809a-47d9-9ed8-fa81b22f3657 K2 2.9523809523809526
7833fafc-39bb-4ea8-8f39-348e9e536390 Venkata Dikshit 2.925
edd78ba4-b452-4816-8f3d-0886cba74f40 Cullen Childress 2.7083333333333335
cd4e688d-9011-470a-8e4b-674cd3b40977 Trishanth Diwate 2.6
25f4a8bf-6e75-46b5-b42c-b4be8808b217 Nisha Yadav 2.4814814814814814
feffe3ee-896f-45c3-9d91-1f6d962c66ce Karthik Muralidharan 2.4468085106382977
c2603593-2d62-4f68-b615-d2cef2145608 mithun 2.44
a366bdbc-ba2f-4b09-ba6c-ad0c614af9ca Arjun Kini 2.353448275862069
2cd90f06-74f3-48cc-922a-cd6b8782ba0f Shubham 2.2666666666666666
85dd59a9-7cf8-49a2-abc4-32b5027a6147 Sai 2.2666666666666666
31a3ba47-6185-4ad9-a

In [248]:
ref_user_kw_dict['d5934fd1-809a-47d9-9ed8-fa81b22f3657']

['default rule',
 'default policy',
 'touched here and leave',
 'stuff functions perspective',
 'error and in order',
 'create role',
 'Land Rover',
 'Lambda',
 'reminder by default',
 'default and click',
 'click on create',
 'step function',
 'Gonna Roll',
 'create our state machine',
 'Lambda function',
 'rolls click',
 'state machine',
 'Aaron',
 'click next not gon',
 'lung functions',
 'target of which Lambda function',
 'production type',
 'stuff is then incident',
 'good visual queue',
 'real time',
 'container is purchase',
 'hear introduction',
 'output from the lambda function',
 'stand on real quick',
 'transaction type',
 'instance where the credit action',
 'position lambda',
 'decision of living',
 'process for production',
 'city for your training figure',
 'link teach',
 'process purpose',
 'business that I kind prepared',
 'cycle function option',
 'set up the time',
 'Token',
 'Coke',
 'process or the exact',
 'return the object',
 'Lambda of function',
 'purchase so

In [245]:
user_kw_dict['e034c7a6-0681-40ba-b7b2-b78de2a2cc3d']

['CSV ether ready',
 'transportation app',
 'animation to switched',
 'HTTP libraries',
 'HTTP',
 'real change white code',
 'second time',
 'offline mode',
 'images just renders',
 'app remember',
 'catch data',
 'jump offline',
 'choose is random',
 'great examples',
 'Lambda',
 'modern application',
 'Lambda functions',
 'lot of different Lambda functions',
 'real change',
 'web component',
 'app Remember',
 'HTTP libraries',
 'second time round',
 'start the app',
 'footer colleagues javascript',
 'couple of differences',
 'chose is a random',
 'Stitches the same styling',
 'web app',
 'HTTP',
 'Switched Off',
 'working hard',
 'deploy by cash',
 'offline mode',
 'core complex pieces',
 'switch up online']